In [1]:
import os
import yaml
import json
import pandas as pd
from pathlib import Path

In [2]:
cwl_dir = '/home/estorrs/pecgs-fusion/cwl/'
Path(cwl_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
sample = m['id']
fq_1 = m['rna-seq']['tumor']['R1']['filepath']
fq_2 = m['rna-seq']['tumor']['R2']['filepath']
cpu = 16
genome_lib_dir = '/data/pecgs/fusion/STAR-Fusion_dependencies/GRCh38_gencode_v37_CTAT_lib_Mar012021.plug-n-play/ctat_genome_lib_build_dir/'
genome_db = '/data/pecgs/fusion/ericscript_dependencies/ericscript_db_homosapiens_ensembl84/'
# ericscript = '/data/pecgs/fusion/ericscript-0.5.5/ericscript.pl'
bwts = '/data/pecgs/fusion/Integrate_dependencies/bwts/'
integrate_fasta = '/data/pecgs/fusion/Integrate_dependencies/STAR/hg38.fa'
integrate_annotations = '/data/pecgs/fusion/Integrate_dependencies/annot.ensembl.GRCh38.txt'
combine_call_script = '/data/pecgs/fusion/Fusion_hg38_scripts/combine_call.pl'
filter_script = '/data/pecgs/fusion/Fusion_hg38_scripts/filter.pl'


In [19]:
template = {
    'fq_1': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'fq_2': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'genome_lib_dir': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'genome_db': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'bwts': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'integrate_executable': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'integrate_fasta': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'integrate_annotations': {
        'class': 'File',
        'path': 'a/file/path'
    },
    'filter_database': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'fusion_annotator_dir': {
        'class': 'Directory',
        'path': 'a/file/path'
    },
    'sample': 'a_string',
    'cpu': 'a_int'
}

In [20]:
yaml.safe_dump(template, open(os.path.join(cwl_dir, 'template.fusion.yaml'), 'w'))

In [25]:
cwl = {
    'class': 'CommandLineTool',
    'cwlVersion': 'v1.0',
    '$namespaces': {'sbg': 'https://www.sevenbridges.com/'},
    'id': 'fusion',
    'label': 'fusion',
    # needs the export for it to work on compute1 with the docker image
    'baseCommand': ['python', '/pecgs-fusion/fusion/fusion.py'],
    'inputs': [
        {
            'id': 'sample',
            'type': 'string',
            'inputBinding': {
                'position': '1'
            }
        },
        {
            'id': 'fq_1',
            'type': 'File',
            'inputBinding': {
                'position': '2'
            }
        },
        {
            'id': 'fq_2',
            'type': 'File',
            'inputBinding': {
                'position': '3'
            }
        },
        {
            'id': 'genome_lib_dir',
            'type': 'Directory',
            'inputBinding': {
                'prefix': '--genome-lib-dir',
                'position': '0'
            }
        },
        {
            'id': 'genome_db',
            'type': 'Directory',
            'inputBinding': {
                'prefix': '--genome-db',
                'position': '0'
            }
        },
        {
            'id': 'bwts',
            'type': 'Directory',
            'inputBinding': {
                'prefix': '--bwts',
                'position': '0'
            }
        },
        {
            'id': 'integrate_executable',
            'type': 'File',
            'inputBinding': {
                'prefix': '--integrate-executable',
                'position': '0'
            }
        },
        {
            'id': 'integrate_fasta',
            'type': 'File',
            'inputBinding': {
                'prefix': '--integrate-fasta',
                'position': '0'
            }
        },
        {
            'id': 'integrate_annotations',
            'type': 'File',
            'inputBinding': {
                'prefix': '--integrate-annotations',
                'position': '0'
            }
        },
        {
            'id': 'filter_database',
            'type': 'Directory',
            'inputBinding': {
                'prefix': '--filter-database',
                'position': '0'
            }
        },
        {
            'id': 'fusion_annotator_dir',
            'type': 'Directory',
            'inputBinding': {
                'prefix': '--fusion-annotator-dir',
                'position': '0'
            }
        },
        {
            'id': 'cpu',
            'type': 'string?',
            'inputBinding': {
                'prefix': '--cpu',
                'position': '0'
            }
        },
        {
            'id': 'environ_PATH',
            'type': 'string?',
            'default': '/miniconda/envs/fusion/bin:$PATH'
        },
        {
            'id': 'environ_LD_LIBRARY_PATH',
            'type': 'string?',
            'default': '/miniconda/envs/fusion/lib/:$LD_LIBRARY_PATH'
        },
    ],
    'arguments': [
        {
            'position': 0,
            'prefix': '--combine-call-script',
            'valueFrom': '/pecgs-fusion/fusion/combine_call.pl'
        },
        {
            'position': 0,
            'prefix': '--filter-script',
            'valueFrom': '/pecgs-fusion/fusion/filter.pl'
        },
    ],
    'outputs': [
        {
            'id': 'filtered_fusions',
            'type': 'File',
            'outputBinding': {'glob': 'Merged_Fusions/Filtered*.tsv'},     
        },
        {
            'id': 'total_fusions',
            'type': 'File',
            'outputBinding': {'glob': 'Merged_Fusions/Total*.tsv'},     
        },
    ],
    'requirements': [
        {
            'class': 'DockerRequirement',
            'dockerPull': 'estorrs/pecgs_fusion:0.0.2'
        },
        {
            'class': 'ResourceRequirement',
            'ramMin': 50000
        },
        {
            'class': 'EnvVarRequirement',
            'envDef': {
                'PATH': '$(inputs.environ_PATH)',
                'LD_LIBRARY_PATH': '$(inputs.environ_LD_LIBRARY_PATH)'
            }
        }
    ]
}

In [26]:
yaml.safe_dump(cwl, open(os.path.join(cwl_dir, 'fusion.cwl'), 'w'))